In [1]:
import soundfile as sf

from transformers import Qwen2_5OmniThinkerForConditionalGeneration, Qwen2_5OmniProcessor
from qwen_omni_utils import process_mm_info

/gscratch/ark/anjo0/miniconda3/envs/taudio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "Qwen/Qwen2.5-Omni-3B"
model = Qwen2_5OmniThinkerForConditionalGeneration.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
processor = Qwen2_5OmniProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.69s/it]
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [3]:
from datasets import load_dataset

ds = load_dataset("gilkeyio/librispeech-alignments")

In [24]:
print(ds['test_clean']['words'][0])

[{'end': 0.66, 'start': 0.5, 'word': 'the'}, {'end': 0.95, 'start': 0.66, 'word': 'dull'}, {'end': 1.41, 'start': 0.95, 'word': 'light'}, {'end': 1.71, 'start': 1.41, 'word': 'fell'}, {'end': 1.94, 'start': 1.71, 'word': 'more'}, {'end': 2.46, 'start': 1.94, 'word': 'faintly'}, {'end': 2.76, 'start': 2.46, 'word': 'upon'}, {'end': 2.84, 'start': 2.76, 'word': 'the'}, {'end': 3.4, 'start': 2.84, 'word': 'page'}, {'end': 3.81, 'start': 3.4, 'word': 'whereon'}, {'end': 4.22, 'start': 3.81, 'word': 'another'}, {'end': 4.85, 'start': 4.22, 'word': 'equation'}, {'end': 5.29, 'start': 4.85, 'word': 'began'}, {'end': 5.48, 'start': 5.29, 'word': 'to'}, {'end': 6.01, 'start': 5.48, 'word': 'unfold'}, {'end': 6.51, 'start': 6.01, 'word': 'itself'}, {'end': 7.1, 'start': 6.51, 'word': 'slowly'}, {'end': 7.72, 'start': 7.53, 'word': 'and'}, {'end': 7.82, 'start': 7.72, 'word': 'to'}, {'end': 8.24, 'start': 7.82, 'word': 'spread'}, {'end': 8.79, 'start': 8.24, 'word': 'abroad'}, {'end': 8.97, 'star

In [ ]:
conversation = [
    # {
    #     "role": "system",
    #     "content": [
    #         {"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}
    #     ],
    # },
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": ds['test_clean']['audio'][0]},
        ],
    },
]

In [16]:
# set use audio in video
USE_AUDIO_IN_VIDEO = True

# Preparation for inference
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios, images, videos = process_mm_info(conversation, use_audio_in_video=USE_AUDIO_IN_VIDEO)
# audios = ds['test_clean']['audio'][0]['array']
# images = None
# videos = None
inputs = processor(text=text, audio=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(model.dtype)

# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, use_audio_in_video=USE_AUDIO_IN_VIDEO)

text = processor.batch_decode(text_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(text)
sf.write(
    "output.wav",
    audio.reshape(-1).detach().cpu().numpy(),
    samplerate=24000,
)

/gscratch/ark/anjo0/miniconda3/envs/taudio/lib/python3.12/site-packages/librosa/core/audio.py:172: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


ValueError: not enough values to unpack (expected 2, got 1)

In [30]:
import soundfile as sf

from transformers import Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor
from qwen_omni_utils import process_mm_info

# default: Load the model on the available device(s)
model = Qwen2_5OmniForConditionalGeneration.from_pretrained("Qwen/Qwen2.5-Omni-3B", torch_dtype="auto", device_map="auto")

# We recommend enabling flash_attention_2 for better acceleration and memory saving.
# model = Qwen2_5OmniForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-Omni-3B",
#     torch_dtype="auto",
#     device_map="auto",
#     attn_implementation="flash_attention_2",
# )

processor = Qwen2_5OmniProcessor.from_pretrained("Qwen/Qwen2.5-Omni-3B")

conversation = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/glass-breaking-151256.mp3"},
            {"type": "text", "text": "Repeat exactly what is said in this audio"},
        ],
    },
]

# set use audio in video
USE_AUDIO_IN_VIDEO = True

# Preparation for inference
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
# audios, images, videos = process_mm_info(conversation, use_audio_in_video=USE_AUDIO_IN_VIDEO)
audios = list(ds['test_clean']['audio'][0]['array'])
images = None
videos = None
inputs = processor(text=text, audio=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(model.dtype)

# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, use_audio_in_video=USE_AUDIO_IN_VIDEO)

text = processor.batch_decode(text_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(text)
sf.write(
    "output.wav",
    audio.reshape(-1).detach().cpu().numpy(),
    samplerate=24000,
)


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.32s/it]
Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


['system\nYou are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.\nuser\nRepeat exactly what is said in this audio\nassistant\nThe dull light fell more faintly upon the page whereon another equation began to unfold itself slowly and to spread abroad its widening tale. If you have any other parts of the audio you want me to repeat or any other questions, feel free to let me know.']


In [27]:
print(ds['test_clean']['audio'][0])

{'path': '1089-134686-0006.flac', 'array': array([-0.00057983, -0.0007019 , -0.0005188 , ..., -0.00067139,
       -0.00027466, -0.00054932]), 'sampling_rate': 16000}
